### Important Note:
Notebook contains huge size of rendered animations and 2D/3D visual files. Due to large size of outputs, we just uploaded source codes of the file. Please refer Github wiki for outputted(full) file. 


# Installing necessity packages

In [ ]:
#!pip install shapely -U
#!pip install lyft-dataset-sdk
#!pip install imagemagick

# Importing Libraries

In [ ]:
import os
import gc
import numpy as np
import pandas as pd

import json
from pprint import pprint
import math
import sys
import time
from datetime import datetime
from typing import Tuple, List
from IPython.display import HTML, Image

import cv2
import matplotlib.pyplot as plt
import sklearn.metrics
from PIL import Image
from collections import defaultdict

from matplotlib.axes import Axes
from matplotlib import animation, rc
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import plot, init_notebook_mode
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

import seaborn as sns
from pyquaternion import Quaternion
from tqdm import tqdm

#Load the  Lyft SDK packages
from lyft_dataset_sdk.utils.map_mask import MapMask
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud
from lyft_dataset_sdk.utils.geometry_utils import view_points, box_in_image, BoxVisibility
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from pathlib import Path

import struct
from abc import ABC, abstractmethod
from functools import reduce
from typing import Tuple, List, Dict
import copy

# Preliminary analysis

In [ ]:
#Let's define working directory 
BASE_PATH = '/home/keceli/3d-data'

In [ ]:
#Checking the directories under the BASE_PATH
dirs = os.listdir(BASE_PATH)
print(dirs)

In [ ]:
#Adjust root folders as an alias paramater.
!ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/test_images images
!ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/test_maps maps
!ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/test_lidar lidar

In [ ]:
#Defining LyftDataset Class
class LyftTestDataset(LyftDataset):
    """Database class for Lyft Dataset to help query and retrieve information from the database."""

    def __init__(self, data_path: str, json_path: str, verbose: bool = True, map_resolution: float = 0.1):
        """Loads database and creates reverse indexes and shortcuts.
        Args:
            data_path: Path to the tables and data.
            json_path: Path to the folder with json files
            verbose: Whether to print status messages during load.
            map_resolution: Resolution of maps (meters).
        """

        self.data_path = Path(data_path).expanduser().absolute()
        self.json_path = Path(json_path)

        self.table_names = [
            "category",
            "attribute",
            "sensor",
            "calibrated_sensor",
            "ego_pose",
            "log",
            "scene",
            "sample",
            "sample_data",
            "map",
        ]

        start_time = time.time()

        # Explicitly assign tables to help the IDE determine valid class members.
        self.category = self.__load_table__("category")
        self.attribute = self.__load_table__("attribute")
        
        
        self.sensor = self.__load_table__("sensor")
        self.calibrated_sensor = self.__load_table__("calibrated_sensor")
        self.ego_pose = self.__load_table__("ego_pose")
        self.log = self.__load_table__("log")
        self.scene = self.__load_table__("scene")
        self.sample = self.__load_table__("sample")
        self.sample_data = self.__load_table__("sample_data")
        
        self.map = self.__load_table__("map")

        if verbose:
            for table in self.table_names:
                print("{} {},".format(len(getattr(self, table)), table))
            print("Done loading in {:.1f} seconds.\n======".format(time.time() - start_time))

        # Initialize LyftDatasetExplorer class
        self.explorer = LyftDatasetExplorer(self)
        # Make reverse indexes for common lookups.
        self.__make_reverse_index__(verbose)
        
    def __make_reverse_index__(self, verbose: bool) -> None:
        """De-normalizes database to create reverse indices for common cases.
        Args:
            verbose: Whether to print outputs.
        """

        start_time = time.time()
        if verbose:
            print("Reverse indexing ...")

        # Store the mapping from token to table index for each table.
        self._token2ind = dict()
        for table in self.table_names:
            self._token2ind[table] = dict()

            for ind, member in enumerate(getattr(self, table)):
                self._token2ind[table][member["token"]] = ind

        # Decorate (adds short-cut) sample_data with sensor information.
        for record in self.sample_data:
            cs_record = self.get("calibrated_sensor", record["calibrated_sensor_token"])
            sensor_record = self.get("sensor", cs_record["sensor_token"])
            record["sensor_modality"] = sensor_record["modality"]
            record["channel"] = sensor_record["channel"]

        # Reverse-index samples with sample_data and annotations.
        for record in self.sample:
            record["data"] = {}
            record["anns"] = []

        for record in self.sample_data:
            if record["is_key_frame"]:
                sample_record = self.get("sample", record["sample_token"])
                sample_record["data"][record["channel"]] = record["token"]

        if verbose:
            print("Done reverse indexing in {:.1f} seconds.\n======".format(time.time() - start_time))

In [ ]:
#Observe dataset features.
classes = ["car", "motorcycle", "bus", "bicycle", "truck", "pedestrian", "other_vehicle", "animal", "emergency_vehicle"]
train_dataset = LyftDataset(data_path='.', json_path='/home/keceli/3d-data/train_data', verbose=True)

In [ ]:
#First five row of the train.csv file.
train = pd.read_csv('/home/keceli/3d-data/train.csv')
sample_submission = pd.read_csv('/home/keceli/3d-data/sample_submission.csv')
print(train.shape)
train.head()

**train.csv** file contains **sample_tokens** in the training set with annotations for all objects, we have the following components:

* **center_x**, **center_y** and **center_z** represent the 3D the world coordinates .
* **width**, **length** and **height** are the dimensions of the observed objects.
* **yaw** is the angle of the volume around the z axis
* **class_name** is the type of object.

In [ ]:
object_columns = ['sample_id', 'object_id', 'center_x', 'center_y', 'center_z',
                  'width', 'length', 'height', 'yaw', 'class_name']
objects = []
for sample_id, ps in tqdm(train.values[:]):
    object_params = ps.split()
    n_objects = len(object_params)
    for i in range(n_objects // 8):
        x, y, z, w, l, h, yaw, c = tuple(object_params[i * 8: (i + 1) * 8])
        objects.append([sample_id, i, x, y, z, w, l, h, yaw, c])
train_objects = pd.DataFrame(
    objects,
    columns = object_columns
)

In [ ]:
numerical_cols = ['object_id', 'center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'yaw']
train_objects[numerical_cols] = np.float32(train_objects[numerical_cols].values)

In [ ]:
train_objects.head()

In [ ]:
#Looking to train_images first 20 files
os.listdir(BASE_PATH + "/train_images")[:20]

In [ ]:
#Exploring the train data
os.listdir(BASE_PATH + "/train_data")

**sample_data.json** contains all the necessity information about the training data. Let's take a look to the informations.

In [ ]:
with open(BASE_PATH + '/train_data/sample_data.json') as f:
    data_json = json.load(f)

print(len(data_json), "records found in sample_data.json")

print("\nBelow showing recorded informations about lidar data:")
pprint(data_json[0])

print('\nBelow showing recored informations about image data:')
pprint(data_json[2])

**scene.json** contains necessity information about the scenes. Let's take a look to the informations.

In [ ]:
with open(BASE_PATH + '/train_data/scene.json') as f:
    scene_json = json.load(f)

print("There are", len(scene_json), "records in sample_data.json")

pprint(scene_json[0])

In [ ]:
for col in object_columns[2:-1]:
    train_objects[col] = train_objects[col].astype('float')
train_objects['confidence'] = 1.0

In [ ]:
train_objects.shape
train_objects.head()
train_objects.describe()

# Visualizing

In [ ]:
fig, ax = plt.subplots(ncols=3,figsize=(10, 10))
sns.distplot(train_objects.center_x, ax = ax[0])
sns.distplot(train_objects.center_y, ax = ax[1])
sns.distplot(train_objects.center_z, ax = ax[2])
plt.suptitle('X, y, z coord distribution')
plt.show();

In the dataset, most of the objects are cars. So that, width, length and height distributions skewed to certain range. 

In [ ]:
fig, ax = plt.subplots(ncols=3,figsize=(10, 10))
sns.distplot(train_objects.width, ax = ax[0])
sns.distplot(train_objects.length, ax = ax[1])
sns.distplot(train_objects.height, ax = ax[2])
plt.suptitle('Width, length, height distribution')
plt.show();

As we said above, cars obejects dominating the other class with the annotated targets.

In [ ]:
class_cnt = train_objects.groupby('class_name').count()[['object_id']].sort_values(by='object_id', ascending=False).reset_index()
class_cnt['p'] = class_cnt.object_id / class_cnt.object_id.sum() 
class_cnt

In [ ]:
train_objects.groupby('class_name').mean()

Submission string

In [ ]:
x, y, z, w, l, h, yaw = train_objects[[
    'center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'yaw']].mean()
mean_prediction_string = ' '.join(map(str, [0.9, x, y, z, 10*w, 10*l, h, yaw, 'car']))

In [ ]:
sample_submission['PredictionString'] = mean_prediction_string 
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
sample_submission.shape
sample_submission.head()

We assume that for below graph, car's cameras are sensing the objects left or right that because small width of the road.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(train_objects['center_x'], color='green', ax=ax).set_title('center_ & and center_y', fontsize=16)
sns.distplot(train_objects['center_y'], color='red', ax=ax).set_title('center_x & center_y', fontsize=16)
plt.xlabel('center_x & center_y', fontsize=15)
plt.show()

The reason could be the camera detect objects either too far ahead or side

In [ ]:
new_train_objects = train_objects.query('class_name == "car"')
plot = sns.jointplot(x=new_train_objects['center_x'][:1000], y=new_train_objects['center_y'][:1000], kind='kde', color='blue')
plot.set_axis_labels('center_x', 'center_y', fontsize=16)
plt.show()

center_z is right skewed and  clustured near -20, and also coordinates are negative because, camera sensor placed at the top of the vehicle. So, most of the times, camera sensor look down to see objects.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(train_objects['center_z'], color='magenta', ax=ax).set_title('center_z', fontsize=16)
plt.xlabel('center_z', fontsize=15)
plt.show()

Yaw value is the volume of z axis

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(train_objects['yaw'], color='darkgreen', ax=ax).set_title('yaw', fontsize=16)
plt.xlabel('yaw', fontsize=15)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(train_objects['width'], color='darkblue', ax=ax).set_title('width', fontsize=16)
plt.xlabel('width', fontsize=15)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(train_objects['length'], color='yellow', ax=ax).set_title('length', fontsize=16)
plt.xlabel('length', fontsize=15)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(train_objects['height'], color='indigo', ax=ax).set_title('height', fontsize=16)
plt.xlabel('height', fontsize=15)
plt.show()

In [ ]:
train_objects['class_name'].value_counts().plot(kind='bar');
plt.title('Class counts');

In [ ]:
train_objects['class_name'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plot = sns.countplot(y="class_name", data=train_objects.query('class_name != "motorcycle" and class_name != "emergency_vehicle" and class_name != "animal"'),
                     palette=['navy', 'darkblue', 'blue', 'dodgerblue', 'skyblue', 'lightblue']).set_title('Object Frequencies', fontsize=16)
plt.yticks(fontsize=14)
plt.xlabel("Count", fontsize=15)
plt.ylabel("Class Name", fontsize=15)
plt.show(plot)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

plot = sns.boxplot(x="class_name", y="center_x",
                   data=train_objects.query('class_name != "motorcycle" and class_name != "emergency_vehicle" and class_name != "animal"'),
                   palette='YlGnBu', ax=ax).set_title('center_x (for different objects)', fontsize=16)

plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel("Class Name", fontsize=15)
plt.ylabel("center_x", fontsize=15)
plt.show(plot)

In [ ]:

fig, ax = plt.subplots(figsize=(15, 15))

plot = sns.boxplot(x="class_name", y="center_y",
                   data=train_objects.query('class_name != "motorcycle" and class_name != "emergency_vehicle" and class_name != "animal"'),
                   palette='YlOrRd', ax=ax).set_title('center_y (for different objects)', fontsize=16)

plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel("Class Name", fontsize=15)
plt.ylabel("center_y", fontsize=15)
plt.show(plot)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

plot = sns.boxplot(x="class_name", y="center_z",
                   data=train_objects.query('class_name != "motorcycle" and class_name != "emergency_vehicle" and class_name != "animal"').query('center_z <= -5'),
                   palette='RdPu', ax=ax).set_title('center_z (for different objects)', fontsize=16)

plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel("Class Name", fontsize=15)
plt.ylabel("center_z", fontsize=15)
plt.show(plot)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

plot = sns.boxplot(x="class_name", y="width",
                   data=train_objects.query('class_name != "motorcycle" and class_name != "emergency_vehicle" and class_name != "animal"'),
                   palette='YlGn', ax=ax).set_title('width (for different objects)', fontsize=16)

plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel("Class Name", fontsize=15)
plt.ylabel("width", fontsize=15)
plt.show(plot)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

plot = sns.boxplot(x="class_name", y="length",
                   data=train_objects.query('class_name != "motorcycle" and class_name != "emergency_vehicle" and class_name != "animal" and length < 15'),
                   palette='Purples', ax=ax).set_title('length (for different objects)', fontsize=16)

plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel("Class Name", fontsize=15)
plt.ylabel("length", fontsize=15)
plt.show(plot)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

plot = sns.boxplot(x="class_name", y="height",
                   data=train_objects.query('class_name != "motorcycle" and class_name != "emergency_vehicle" and class_name != "animal" and height < 6'),
                   palette='Reds', ax=ax).set_title('height (for different objects)', fontsize=16)

plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel("Class Name", fontsize=15)
plt.ylabel("height", fontsize=15)
plt.show(plot)

# Animating 2D/3D

Metadata of the scene

In [ ]:
train_dataset.list_scenes()

In [ ]:
palo_alto = train_dataset.scene[0]
first_sample_token = palo_alto["first_sample_token"]
train_dataset.render_sample(first_sample_token)

In [ ]:
my_sample = train_dataset.get('sample', first_sample_token)
my_sample

In [ ]:
train_dataset.render_pointcloud_in_image(sample_token=first_sample_token)

We will project LiDAR and Camera data into one image.

In [ ]:
sensor = 'CAM_FRONT'
cam_front = train_dataset.get('sample_data', my_sample['data'][sensor])

In [ ]:
train_dataset.render_sample_data(cam_front['token'], with_anns=False)

In [ ]:
train_dataset.render_pointcloud_in_image(sample_token = my_sample["token"],
                                      dot_size = 1,
                                      camera_channel = 'CAM_BACK')

Let's look at a particular sensor of sample_data

In [ ]:
sensor_channel = 'CAM_FRONT'
my_sample_data = train_dataset.get('sample_data', my_sample['data'][sensor_channel])

In [ ]:
train_dataset.render_sample_data(my_sample_data['token'])

In [ ]:
train_dataset.render_sample_data(my_sample_data['token'], with_anns=False)

Let's render different annotation

In [ ]:
my_ann_token = my_sample['anns'][25]
my_ann =  my_sample_data.get('sample_annotation', my_ann_token)

In [ ]:
train_dataset.render_annotation(my_ann)

We will visualize lidar data

In [ ]:
axes_limits = [
    [-30, 50], # X axis range
    [-30, 20], # Y axis range
    [-3, 10]   # Z axis range
]
axes_str = ['X', 'Y', 'Z']

def display_frame_statistics(lidar_points, points=0.2):
    """
    Displays statistics for a single frame. Draws 3D plot of the lidar point cloud data and point cloud
    projections to various planes.
    
    Parameters
    ----------
    lidar_points: lidar data points 
    points          : Fraction of lidar points to use. Defaults to `0.2`, e.g. 20%.
    """
    
    points_step = int(1. / points)
    point_size = 0.01 * (1. / points)
    pc_range = range(0, lidar_points.shape[1], points_step)
    pc_frame = lidar_points[:, pc_range]
    def draw_point_cloud(ax, title, axes=[0, 1, 2]):
        """Convenient method for drawing various point cloud projections as a part of frame statistics"""
        ax.set_facecolor('black')
        ax.grid(False)
        ax.scatter(*pc_frame[axes, :], s=point_size, c='white', cmap='grey')
        if len(axes) == 3: # 3D configs
            text_color = 'white'
            ax.set_xlim3d([-10, 30])
            ax.set_ylim3d(*axes_limits[axes[1]])
            ax.set_zlim3d(*axes_limits[axes[2]])
            ax.set_zlabel('{} axis'.format(axes_str[axes[2]]), color='white')
        else: # 2D configs
            text_color = 'black' # the `figure` is white
            ax.set_xlim(*axes_limits[axes[0]])
            ax.set_ylim(*axes_limits[axes[1]])
        ax.set_title(title, color=text_color)
        ax.set_xlabel('{} axis'.format(axes_str[axes[0]]), color=text_color)
        ax.set_ylabel('{} axis'.format(axes_str[axes[1]]), color=text_color)
            
    # Draw point cloud data as 3D plot
    f2 = plt.figure(figsize=(15, 8))
    ax2 = f2.add_subplot(111, projection='3d')
    # make the panes transparent
    ax2.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax2.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax2.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    draw_point_cloud(ax2, '3D plot')
    
    plt.show()
    plt.close(f2)
    # Draw point cloud data as plane projections
    f, ax3 = plt.subplots(3, 1, figsize=(15, 25))
#     f.set_facecolor('black')
    draw_point_cloud(
        ax3[0], 
        'XZ projection (Y = 0)', #, the car is moving in direction left to right', ?
        axes=[0, 2] # X and Z axes
    )
    draw_point_cloud(
        ax3[1], 
        'XY projection (Z = 0)', #, the car is moving in direction left to right',? 
        axes=[0, 1] # X and Y axes
    )
    draw_point_cloud(
        ax3[2], 
        'YZ projection (X = 0)', #, the car is moving towards the graph plane', ?
        axes=[1, 2] # Y and Z axes
    )
    plt.show()
    plt.close(f)

In [ ]:
lidar_top = train_dataset.get('sample_data', my_sample['data']['LIDAR_TOP'])
pc = LidarPointCloud.from_file(Path(lidar_top['filename']))

In [ ]:
display_frame_statistics(pc.points, points=0.5)

In [ ]:
train_dataset.render_sample_data(lidar_top['token'], with_anns=False)

In [ ]:
train_dataset.render_sample_data(lidar_top['token'], with_anns=False)

Let's store lidar and image data in seperate frames

In [ ]:
lidar_dataset = []
image_dataset = []

for data in data_json:
    if data['fileformat'] == 'bin':
        lidar_dataset.append(data)
    else:
        image_dataset.append(data)

In [ ]:
lidar_df = pd.DataFrame(lidar_dataset)
image_df = pd.DataFrame(image_dataset)

print(lidar_df.shape)
print(image_df.shape)

In [ ]:
lidar_df.head()

In [ ]:
image_df.head()


In [ ]:
# we will augment image_df which will provide information about camera and host
image_df['host'] = image_df['filename'].apply(lambda st: st.strip('images/host-').split('_')[0])
image_df['cam'] = image_df['filename'].apply(lambda st: st.split('_')[1])
image_df['timestamp'] = image_df['filename'].apply(lambda st: st.split('_')[2].strip('.jpeg'))

In [ ]:
image_df.head()


In [ ]:
#We will store frames which we had created.
image_df.to_csv("/home/keceli/3d-data/sample_data_images.csv")
lidar_df.to_csv("/home/keceli/3d-data/lidar_data_images.csv")

Let's explore image data

In [ ]:
image_df['host'].value_counts()


In [ ]:
image_df['cam'].value_counts()


In [ ]:
def display_host_sample(host, n_images, jumps=1):
    cams = list(sorted(image_df['cam'].unique()))
    
    fig, axs = plt.subplots(
        n_images, len(cams), figsize=(3*len(cams), 3*n_images), 
        sharex=True, sharey=True, gridspec_kw = {'wspace':0.1, 'hspace':0.1}
    )
    
    for i in range(n_images):
        for c, cam in enumerate(cams):
            if i == 0:
                axs[i, c].set_title(cam)
            
            mask1 = image_df.cam == cam
            mask2 = image_df.host == host
            image_path = image_df[mask1 & mask2]
            image_path = image_path.sort_values('timestamp')['filename'].iloc[i*jumps]
            
            img = cv2.imread('/home/keceli/3d-data/train_' + image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (200, 200))
            
            axs[i, c].imshow(img)
            axs[i, c].axis('off')

In [ ]:
display_host_sample('005', 5, jumps=1)

In [ ]:
display_host_sample('101', 5, jumps=1) 

In [ ]:
display_host_sample('009', 5, jumps=1) 

In [ ]:
def generate_next_token(scene):
    scene = train_dataset.scene[scene]
    sample_token = scene['first_sample_token']
    sample_record = train_dataset.get("sample", sample_token)
    
    while sample_record['next']:
        sample_token = sample_record['next']
        sample_record = train_dataset.get("sample", sample_token)
        
        yield sample_token

def animate_images(scene, frames, pointsensor_channel='LIDAR_TOP', interval=1):
    cams = [
        'CAM_FRONT',
        'CAM_FRONT_RIGHT',
        'CAM_BACK_RIGHT',
        'CAM_BACK',
        'CAM_BACK_LEFT',
        'CAM_FRONT_LEFT',
    ]

    generator = generate_next_token(scene)

    fig, axs = plt.subplots(
        2, len(cams), figsize=(3*len(cams), 6), 
        sharex=True, sharey=True, gridspec_kw = {'wspace': 0, 'hspace': 0.1}
    )
    
    plt.close(fig)

    def animate_fn(i):
        for _ in range(interval):
            sample_token = next(generator)
            
        for c, camera_channel in enumerate(cams):    
            sample_record = train_dataset.get("sample", sample_token)

            pointsensor_token = sample_record["data"][pointsensor_channel]
            camera_token = sample_record["data"][camera_channel]
            
            axs[0, c].clear()
            axs[1, c].clear()
            
            train_dataset.render_sample_data(camera_token, with_anns=False, ax=axs[0, c])
            train_dataset.render_sample_data(camera_token, with_anns=True, ax=axs[1, c])
            
            axs[0, c].set_title("")
            axs[1, c].set_title("")

    anim = animation.FuncAnimation(fig, animate_fn, frames=frames, interval=interval)
    
    return anim

In [ ]:
%%time
anim = animate_images(scene=0, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
anim.save('/home/keceli/3d-data/7.gif', fps = 8)

In [ ]:
anim = animate_images(scene=10, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
anim = animate_images(scene=50, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
anim = animate_images(scene=100, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

Animate Lidar

In [ ]:
def animate_lidar(scene, frames, pointsensor_channel='LIDAR_TOP', with_anns=True, interval=1):
    generator = generate_next_token(scene)

    fig, axs = plt.subplots(1, 1, figsize=(8, 8))
    plt.close(fig)

    def animate_fn(i):
        for _ in range(interval):
            sample_token = next(generator)
        
        axs.clear()
        sample_record = train_dataset.get("sample", sample_token)
        pointsensor_token = sample_record["data"][pointsensor_channel]
        train_dataset.render_sample_data(pointsensor_token, with_anns=with_anns, ax=axs)

    anim = animation.FuncAnimation(fig, animate_fn, frames=frames, interval=interval)
    
    return anim

In [ ]:
%%time
anim = animate_lidar(scene=0, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
%%time
anim = animate_lidar(scene=10, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
anim = animate_lidar(scene=50, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
%%time
anim = animate_lidar(scene=100, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
Image(url='/home/keceli/3d-data/a.gif')